## Predicting Survival on the Titanic

### History
Perhaps one of the most infamous shipwrecks in history, the Titanic sank after colliding with an iceberg, killing 1502 out of 2224 people on board. Interestingly, by analysing the probability of survival based on few attributes like gender, age, and social status, we can make very accurate predictions on which passengers would survive. Some groups of people were more likely to survive than others, such as women, children, and the upper-class. Therefore, we can learn about the society priorities and privileges at the time.

### Assignment:

Build a Machine Learning Pipeline, to engineer the features in the data set and predict who is more likely to Survive the catastrophe.

Follow the Jupyter notebook below, and complete the missing bits of code, to achieve each one of the pipeline steps.

In [173]:
import re

# to handle datasets
import pandas as pd
import numpy as np

# for visualization
import matplotlib.pyplot as plt

# to divide train and test set
from sklearn.model_selection import train_test_split

# feature scaling
from sklearn.preprocessing import StandardScaler

# to build the models
from sklearn.linear_model import LogisticRegression

# to evaluate the models
from sklearn.metrics import accuracy_score, roc_auc_score

# to persist the model and the scaler
import joblib

# to visualise al the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)

## Prepare the data set

In [174]:
# load the data - it is available open source and online

data = pd.read_csv('phpMYEkMl.csv')

# display data
data.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29,0,0,24160,211.3375,B5,S,2,?,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.55,C22 C26,S,11,?,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2,1,2,113781,151.55,C22 C26,S,?,?,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30,1,2,113781,151.55,C22 C26,S,?,135,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25,1,2,113781,151.55,C22 C26,S,?,?,"Montreal, PQ / Chesterville, ON"


In [175]:
# replace interrogation marks by NaN values

data = data.replace('?', np.nan)

In [176]:
# retain only the first cabin if more than
# 1 are available per passenger

def get_first_cabin(row):
    try:
        return row.split()[0]
    except:
        return np.nan
    
data['cabin'] = data['cabin'].apply(get_first_cabin)

In [177]:
# extracts the title (Mr, Ms, etc) from the name variable

def get_title(passenger):
    line = passenger
    if re.search('Mrs', line):
        return 'Mrs'
    elif re.search('Mr', line):
        return 'Mr'
    elif re.search('Miss', line):
        return 'Miss'
    elif re.search('Master', line):
        return 'Master'
    else:
        return 'Other'
    
data['title'] = data['name'].apply(get_title)

In [178]:
# cast numerical variables as floats

data['fare'] = data['fare'].astype('float')
data['age'] = data['age'].astype('float')

In [179]:
# drop unnecessary variables

data.drop(labels=['name','ticket', 'boat', 'body','home.dest'], axis=1, inplace=True)

# display data
data.head()

,pclass,survived,sex,age,sibsp,parch,fare,cabin,embarked,title
0,1,1,female,29.0000,0,0,211.3375,B5,S,Miss
1,1,1,male,0.9167,1,2,151.5500,C22,S,Master
2,1,0,female,2.0000,1,2,151.5500,C22,S,Miss
3,1,0,male,30.0000,1,2,151.5500,C22,S,Mr
4,1,0,female,25.0000,1,2,151.5500,C22,S,Mrs


In [180]:
# save the data set

data.to_csv('titanic.csv', index=False)

## Data Exploration

### Find numerical and categorical variables

In [238]:
target = 'survived'
df = pd.read_csv('titanic.csv')
df.head()

,pclass,survived,sex,age,sibsp,parch,fare,cabin,embarked,title
0,1,1,female,29.0000,0,0,211.3375,B5,S,Miss
1,1,1,male,0.9167,1,2,151.5500,C22,S,Master
2,1,0,female,2.0000,1,2,151.5500,C22,S,Miss
3,1,0,male,30.0000,1,2,151.5500,C22,S,Mr
4,1,0,female,25.0000,1,2,151.5500,C22,S,Mrs


In [239]:
vars_num = [c for c in df.columns if not df[c].dtype == 'O']

vars_cat = [c for c in df.columns if df[c].dtype == 'O']

print('Number of numerical variables: {}'.format(len(vars_num)))
print('Number of categorical variables: {}'.format(len(vars_cat)))

Number of numerical variables: 6
Number of categorical variables: 4


### Find missing values in variables

In [240]:
df[vars_num].isnull().sum()

pclass        0
survived      0
age         263
sibsp         0
parch         0
fare          1
dtype: int64

In [241]:
df[vars_cat].isnull().sum()


sex            0
cabin       1014
embarked       2
title          0
dtype: int64

### Determine cardinality of categorical variables

In [242]:
df[vars_cat].nunique()

sex           2
cabin       181
embarked      3
title         5
dtype: int64

### Determine the distribution of numerical variables

In [243]:
df[vars_num].describe()

,pclass,survived,age,sibsp,parch,fare
count,1309.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000
mean,2.294882,0.381971,29.881135,0.498854,0.385027,33.295479
std,0.837836,0.486055,14.413500,1.041658,0.865560,51.758668
min,1.000000,0.000000,0.166700,0.000000,0.000000,0.000000
25%,2.000000,0.000000,21.000000,0.000000,0.000000,7.895800
50%,3.000000,0.000000,28.000000,0.000000,0.000000,14.454200
75%,3.000000,1.000000,39.000000,1.000000,0.000000,31.275000
max,3.000000,1.000000,80.000000,8.000000,9.000000,512.329200


## Separate data into train and test

Use the code below for reproducibility. Don't change it.

In [244]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop('survived', axis=1),  # predictors
    data['survived'],  # target
    test_size=0.2,  # percentage of obs in test set
    random_state=0)  # seed to ensure reproducibility

X_train.shape, X_test.shape

((1047, 9), (262, 9))

## Feature Engineering

### Extract only the letter (and drop the number) from the variable Cabin

In [245]:
X_train['cabin'] = X_train['cabin'].str.extract(r'(^\w{1})')
X_test['cabin'] = X_test['cabin'].str.extract(r'(^\w{1})')
X_train.head()

,pclass,sex,age,sibsp,parch,fare,cabin,embarked,title
1118,3,male,25.0,0,0,7.9250,NaN,S,Mr
44,1,female,41.0,0,0,134.5000,E,C,Miss
1072,3,male,NaN,0,0,7.7333,NaN,Q,Mr
1130,3,female,18.0,0,0,7.7750,NaN,S,Miss
574,2,male,29.0,1,0,21.0000,NaN,S,Mr


### Fill in Missing data in numerical variables:

- Add a binary missing indicator
- Fill NA in original variable with the median

In [246]:
vars_num.remove(target)

In [247]:
for c in vars_num:
    X_train[c] = X_train[c].fillna(X_train[c].median())
    X_test[c] = X_test[c].fillna(X_train[c].median())
    
X_train[vars_num].isnull().sum()

pclass    0
age       0
sibsp     0
parch     0
fare      0
dtype: int64

### Replace Missing data in categorical variables with the string **Missing**

In [248]:
for c in vars_cat:
    X_train[c] = X_train[c].fillna('Missing')
    X_test[c] = X_test[c].fillna('Missing')
    
X_train[vars_cat].isnull().sum()

sex         0
cabin       0
embarked    0
title       0
dtype: int64

### Remove rare labels in categorical variables

- remove labels present in less than 5 % of the passengers

In [249]:
def find_frequent_labels(df, var, rare_perc):
    df = df.copy()
    tmp = df.groupby(var).count() / len(df)
    return tmp[tmp>rare_perc].index
            

In [250]:
for c in vars_cat:
    frequent_labels = find_frequent_labels(X_train, c, .05)
    X_train[c] = np.where(X_train[c].isin(frequent_labels), X_train[c], 'Rare')
    X_test[c] = np.where(X_test[c].isin(frequent_labels), X_test[c], 'Rare')

### Perform one hot encoding of categorical variables into k-1 binary variables

- k-1, means that if the variable contains 9 different categories, we create 8 different binary variables
- Remember to drop the original categorical variable (the one with the strings) after the encoding

In [251]:
for c in vars_cat:
    X_train = pd.concat([X_train, pd.get_dummies(X_train[c], prefix=c+'_')], axis=1).drop(columns=[c])
    X_test = pd.concat([X_test, pd.get_dummies(X_test[c], prefix=c+'_')], axis=1).drop(columns=[c])


In [252]:

for idx, var in enumerate(X_train.columns):
    if var not in X_test.columns:
        X_test.insert(loc=idx, column=var, value=0)

In [253]:
X_test.head()

,pclass,age,sibsp,parch,fare,sex__female,sex__male,cabin__A,cabin__B,cabin__C,cabin__D,cabin__E,cabin__F,cabin__G,cabin__Missing,cabin__T,embarked__C,embarked__Missing,embarked__Q,embarked__S,title__Master,title__Miss,title__Mr,title__Mrs,title__Other
1139,3,38.0,0,0,7.8958,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0
533,2,21.0,0,1,21.0000,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0
459,2,42.0,1,0,27.0000,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0
1150,3,28.0,0,0,14.5000,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0
393,2,25.0,0,0,31.5000,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0


In [254]:
X_train.head()

,pclass,age,sibsp,parch,fare,sex__female,sex__male,cabin__A,cabin__B,cabin__C,cabin__D,cabin__E,cabin__F,cabin__G,cabin__Missing,cabin__T,embarked__C,embarked__Missing,embarked__Q,embarked__S,title__Master,title__Miss,title__Mr,title__Mrs,title__Other
1118,3,25.0,0,0,7.9250,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0
44,1,41.0,0,0,134.5000,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0
1072,3,28.0,0,0,7.7333,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0
1130,3,18.0,0,0,7.7750,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0
574,2,29.0,1,0,21.0000,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0


### Scale the variables

- Use the standard scaler from Scikit-learn

In [255]:
X_train.columns

Index(['pclass', 'age', 'sibsp', 'parch', 'fare', 'sex__female', 'sex__male',
       'cabin__A', 'cabin__B', 'cabin__C', 'cabin__D', 'cabin__E', 'cabin__F',
       'cabin__G', 'cabin__Missing', 'cabin__T', 'embarked__C',
       'embarked__Missing', 'embarked__Q', 'embarked__S', 'title__Master',
       'title__Miss', 'title__Mr', 'title__Mrs', 'title__Other'],
      dtype='object')

In [256]:
X_test.columns

Index(['pclass', 'age', 'sibsp', 'parch', 'fare', 'sex__female', 'sex__male',
       'cabin__A', 'cabin__B', 'cabin__C', 'cabin__D', 'cabin__E', 'cabin__F',
       'cabin__G', 'cabin__Missing', 'cabin__T', 'embarked__C',
       'embarked__Missing', 'embarked__Q', 'embarked__S', 'title__Master',
       'title__Miss', 'title__Mr', 'title__Mrs', 'title__Other'],
      dtype='object')

In [257]:
from sklearn.preprocessing import StandardScaler

In [258]:
scaler = StandardScaler()
scaler.fit(X=X_train)

# X_train = pd.concat([X_train[vars_cat].reset_index(drop=True),
#                      pd.DataFrame(scaler.transform(X_train[vars_num]), columns=vars_num)], 
#                      axis=1)
# X_test = pd.concat([X_test[vars_cat].reset_index(drop=True),
#                      pd.DataFrame(scaler.transform(X_test[vars_num]), columns=vars_num)], 
#                      axis=1)

X_train = pd.DataFrame(scaler.transform(X_train))

In [259]:
X_test = pd.DataFrame(scaler.transform(X_test))

## Train the Logistic Regression model

- Set the regularization parameter to 0.0005
- Set the seed to 0

In [260]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(C=0.0005, random_state=0)
model.fit(X_train, y_train)

LogisticRegression(C=0.0005, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

## Make predictions and evaluate model performance

Determine:
- roc-auc
- accuracy

**Important, remember that to determine the accuracy, you need the outcome 0, 1, referring to survived or not. But to determine the roc-auc you need the probability of survival.**

In [268]:
print("baseline:", sum(y_train) / len(y_train))
1 - sum(y_test - model.predict(X_test)) / len(y_test) # accuracy

baseline: 0.38204393505253104


0.7251908396946565

That's it! Well done

**Keep this code safe, as we will use this notebook later on, to build production code, in our next assignement!!**